In [ ]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

In [ ]:
spark.sql('''
SELECT
    A.D,
    SUM(COUNT(DISTINCT B.USER_ID)) OVER (ORDER BY A.DATE RANGE BETWEEN 7 PRECEDING AND CURRENT ROW) AS USER_7D,
    SUM(COUNT(DISTINCT B.USER_ID)) OVER (ORDER BY A.DATE RANGE BETWEEN 28 PRECEDING AND CURRENT ROW) AS USER_28D,
    FROM_UTC_TIMESTAMP(NOW(), 'GMT+7') AS LAST_EXECUTED_TIME
FROM delta.`s3a://silver/CALENDAR_DIM` A 
LEFT JOIN delta.`s3a://silver/USER_RATING_FACT` B ON A.DATE = TO_DATE(B.CREATED_AT)
GROUP BY A.DATE
ORDER BY A.DATE
''').write.format('delta').mode('overwrite').save('s3a://gold/USER_RATING_7D_28D')

In [ ]:
spark.stop()